# Introduction of NMF for recommendation systems

SLIDES -> [please click](https://docs.google.com/presentation/d/119Foj6P_X2t1RxCW1TFvrYI-pMOyV2kQgUgHSvccya0/edit?usp=sharing)

# Prepare Data

In [1]:
!gdown --id 1woyznL9rtKIALYdfKulYyxtjetcST4HU
!gdown --id 1mCwvXi4P0LjNHi_H-B9QSXHXopKPrM7d
!gdown --id 1dk-CILCnNi7Q3YrJEcjmuxRHLxBmNdKF

Downloading...
From: https://drive.google.com/uc?id=1woyznL9rtKIALYdfKulYyxtjetcST4HU
To: /content/movie_rating.zip
100% 675k/675k [00:00<00:00, 44.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mCwvXi4P0LjNHi_H-B9QSXHXopKPrM7d
To: /content/movie_meta.zip
12.8MB [00:00, 78.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dk-CILCnNi7Q3YrJEcjmuxRHLxBmNdKF
To: /content/movie_link_small.csv
100% 183k/183k [00:00<00:00, 58.2MB/s]


In [2]:
!unzip movie_rating.zip
!unzip movie_meta.zip

Archive:  movie_rating.zip
  inflating: ratings_small.csv       
Archive:  movie_meta.zip
  inflating: movies_metadata.csv     


In [41]:
import numpy as np
import pandas as pd
import regex as re

In [11]:
moviemeta = pd.read_csv('movies_metadata.csv')
moviemeta

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,Fa

In [9]:
ratingdat = pd.read_csv('ratings_small.csv')
ratingdat

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [10]:
linkdat = pd.read_csv('movie_link_small.csv')
linkdat

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9120,162672,3859980,402672.0
9121,163056,4262980,315011.0
9122,163949,2531318,391698.0
9123,164977,27660,137608.0


In [7]:
moviedat = linkdat.join(moviemeta,on='tmdbId',how='left')
moviedat

,movieId,imdbId,tmdbId,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,1,114709,862.0,False,NaN,0,[],NaN,101230,tt0109001,nl,06,"Sarah, 30, single, well educated, likes art, p...",0.062128,/xnsvk2a8BbvGfnFdynKSuA4qVFq.jpg,[],[],1994-09-01,0.0,0.0,"[{'iso_639_1': 'nl', 'name': 'Nederlands'}, {'...",Released,NaN,1-900,False,5.0,1.0
1,2,113497,8844.0,False,NaN,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",NaN,48677,tt0087065,en,Cloak & Dagger,11-year-old Davey's mother is dead and his fat...,9.73502,/8dmtQGw1IHbs8V93EWrfbwtLgi6.jpg,"[{'name': 'Universal Pictures', 'id': 33}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1984-08-10,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,You can cry 'wolf' once too often!,Cloak & Dagger,False,6.2,40.0
2,3,113228,15602.0,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 99, 'name...",http://www.the2escobars.com,53190,tt2700330,en,The Two Escobars,"Pablo Escobar was the richest, most powerful d...",0.758472,/ctmHQio07EMng7dtUvaG8ZLxawk.jpg,[],[],2010-06-22,0.0,100.0,[],Released,Sometimes it really is a matter of life and de...,The Two Escobars,False,7.6,14.0
3,4,114885,31357.0,False,NaN,0,"[{'id': 10770, 'name': 'TV Movie'}, {'id': 18,...",NaN,137310,tt0472576,en,Wild Hearts,"When Bob Hart, a widowed police detective tryi...",1.05851,/fLl99MgxsumFK9gTyzzkxDjMJJE.jpg,"[{'name': 'Larry Levinson Productions', 'id': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",2006-07-08,0.0,84.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Wild Hearts,False,5.0,4.0
4,5,113041,11862.0,False,"{'id': 325798, 'name': 'Star Crash Collection'...",0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 878, 'n...",NaN,22049,tt0079946,en,Starcrash,A pair of smugglers manage to pick up a castaw...,1.98658,/hOx1RiBsqNWaKnFelJ9wWk7fAJa.jpg,"[{'name': 'New World Pictures', 'id': 1950}, {...","[{'iso_3166_1': 'IT', 'name': 'Italy'}, {'iso_...",1978-12-21,0.0,94.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A galactic adventure beyond your wildest dreams!,Starcrash,False,4.3,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,162672,3859980,402672.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9121,163056,4262980,315011.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9122,163949,2531318,391698.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9123,164977,27660,137608.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Create User X Rating matrix

In [12]:
cmat = pd.crosstab(ratingdat['userId'],ratingdat['movieId'],ratingdat['rating'],aggfunc=sum)

In [13]:
cmat

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,...,155820,156025,156387,156607,156609,156726,157200,157296,157407,157667,158238,158314,158528,158956,159093,159462,159690,159755,159858,159972,160080,160271,160438,160440,160563,160565,160567,160590,160656,160718,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
cmat = cmat.fillna(0)
cmat

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,...,155820,156025,156387,156607,156609,156726,157200,157296,157407,157667,158238,158314,158528,158956,159093,159462,159690,159755,159858,159972,160080,160271,160438,160440,160563,160565,160567,160590,160656,160718,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Decompose Matrix into two matrices using NMF

In [15]:
from sklearn.decomposition import NMF # Non negative matrix factorization
nmf = NMF(100)
nmf.fit(cmat)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=100, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [16]:
H = pd.DataFrame(np.round(nmf.components_,2), columns=cmat.columns)
H

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,...,155820,156025,156387,156607,156609,156726,157200,157296,157407,157667,158238,158314,158528,158956,159093,159462,159690,159755,159858,159972,160080,160271,160438,160440,160563,160565,160567,160590,160656,160718,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
0,8.96,6.31,4.30,0.00,0.00,1.37,0.00,0.00,0.00,0.14,4.61,0.00,0.00,0.00,0.00,4.15,1.61,0.00,2.64,0.00,1.75,0.00,0.00,0.00,1.42,0.0,0.00,0.00,0.00,0.0,0.00,0.00,7.69,0.0,2.59,0.0,0.0,2.32,0.00,0.00,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.95,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.28,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,1.67,0.00,0.00,0.00,0.00,0.0,0.00,0.08,0.21,0.0,0.00,0.07,1.99,0.0,0.00,0.0,0.0,2.21,0.04,0.01,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.32,1.85,0.15,0.08,0.28,0.53,0.09,0.06,0.12,2.83,1.24,0.07,0.07,0.00,0.14,0.00,0.16,0.00,1.37,0.21,1.05,0.78,0.50,0.12,0.00,0.0,0.22,0.00,0.00,0.0,0.86,0.93,1.68,0.0,0.00,0.0,0.0,1.90,0.04,0.00,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.38,0.00,2.68,1.75,1.81,0.00,0.29,0.00,0.32,0.01,2.67,0.34,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.03,0.00,0.00,0.0,0.03,0.00,0.00,0.0,0.00,0.0,0.0,3.88,0.00,0.01,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.35,0.00,0.00,0.00,0.00,0.25,0.20,0.00,0.00,0.00,0.30,0.00,0.00,0.35,0.00,0.40,0.50,0.00,0.00,0.00,0.40,0.00,0.00,0.00,0.45,0.0,0.00,0.00,0.00,0.0,0.00,0.40,0.40,0.0,0.40,0.0,0.0,0.30,0.00,0.00,...,0.0,0.0,0.4,0.00,0.0,0.0,0.0,0.3,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.35,0.4,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.03,0.22,0.21,0.00,0.25,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.30,0.00,0.00,0.07,0.01,0.00,0.00,0.00,0.00,0.03,0.0,0.03,0.00,0.00,0.0,0.02,0.28,0.17,0.0,0.00,0.0,0.0,0.22,0.00,0.00,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.15,0.00,0.00,0.02,0.00,0.05,0.21,0.00,0.00,0.00,0.16,0.00,0.00,0.00,0.00,0.00,0.27,0.01,0.00,0.00,0.03,0.02,0.00,0.10,0.18,0.0,0.00,0.00,0.04,0.0,0.02,0.16,0.03,0.0,0.04,0.0,0.0,0.21,0.00,0.00,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.05,0.26,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.05,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.01,0.13,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.01,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.15,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.18,0.00,0.00,0.11,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.37,0.0,0.00,0.03,0.00,0.0,0.00,0.00,0.12,0.0,0.35,0.0,0.0,0.04,0.00,0.02,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0

In [17]:
W = pd.DataFrame(np.round(nmf.transform(cmat),2), columns=H.index)
W

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.01,0.00,0.00,0.01,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.04,0.0,0.0,0.0,0.01,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.01,0.0,0.00,0.00,0.0,0.02,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.10,0.0,0.00,0.00,0.00,0.11,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0
1,0.00,0.00,0.77,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.02,0.0,0.03,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.94,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.0,0.03,0.00,0.00,0.0
2,0.00,0.00,0.10,0.00,0.0,0.03,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.03,0.0,0.00,0.0,0.0,0.0,0.00,0.16,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.19,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.03,0.0,0.00,0.0,0.00,0.0,0.28,0.14,0.0,0.00,0.17,0.08,0.0,0.00,0.0,0.11,0.0,0.0,0.00,0.0,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0
3,0.03,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.89,0.0,0.0,0.01,0.00,0.00,0.00,1.03,0.0,0.0,0.00,0.0,0.02,0.0,0.0,0.0,0.12,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.00,0.00,0.0,0.28,0.00,0.00,0.0,0.00,0.0,0.02,0.0,0.00,0.00,0.0,0.03,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.78,0.0,0.00,0.19,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0
4,0.09,0.00,0.00,0.01,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.05,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.06,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.00,0.00,0.2,0.00,0.02,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.53,0.0,0.00,1.07,0.29,0.0,0.00,0.1,0.37,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.10,0.00,0.0,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,0.00,0.00,0.40,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.10,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.00,0.00,0.0,0.12,0.00,0.00,0.0,1.46,0.0,0.02,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.13,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0
667,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.01,0.00,0.01,0.01,0.03,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.24,...,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.01,0.00,0.0,0.06,0.0,0.00,0.0,0.00,0.00,0.1,0.00,0.00,0.00,0.0,0.04,0.0,0.03,0.0,0.0,0.00,0.0,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0
668,0.00,0.02,0.00,0.01,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.01,0.00,0.02,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.03,0.0,0.00,0.0,0.0,0.0,0.00,0.59,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.00,0.00,0.0,0.05,0.00,0.00,0.0,0.02,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.0,0.00,0.02,0.00,0.0
669,0.00,0.00,0.06,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.08,0.03,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.04,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.19,...,0.0,0.0,0.0,0.03,0.00,0.0,0.00,0.00,0.00,0.0,0.38,0.0,0.00,0.0,0.04,0.00,0.0,0.00,0.11,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.15,0.00,0.00,0.00,0.13,0.00,0.08,0.0,0.00,0.00,0.07,0.0


In [18]:
reconstructed = pd.DataFrame(np.round(np.dot(W,H),2), columns=cmat.columns)
reconstructed.index = cmat.index
reconstructed

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,...,155820,156025,156387,156607,156609,156726,157200,157296,157407,157667,158238,158314,158528,158956,159093,159462,159690,159755,159858,159972,160080,160271,160438,160440,160563,160565,160567,160590,160656,160718,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.04,0.05,0.00,0.00,0.00,0.08,0.01,0.00,0.02,0.02,0.08,0.00,0.00,0.01,0.00,0.04,0.03,0.00,0.04,0.00,0.07,0.00,0.00,0.03,0.06,0.00,0.00,0.03,0.03,0.00,0.01,0.04,0.11,0.01,0.07,0.0,0.00,0.06,0.00,0.01,...,0.0,0.00,0.0,0.00,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0
2,0.97,1.43,0.12,0.12,0.24,0.87,0.10,0.05,0.09,2.19,1.27,0.07,0.05,0.24,0.11,0.68,1.17,0.39,1.07,0.16,1.45,0.83,0.38,0.15,1.08,0.18,0.17,0.44,0.66,0.44,0.76,1.70,2.12,0.56,1.04,0.0,0.01,1.87,0.08,0.39,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,0.67,0.37,0.03,0.03,0.14,0.23,0.06,0.03,0.05,0.40,0.28,0.01,0.01,0.00,0.02,0.30,0.09,0.01,0.24,0.02,0.14,0.11,0.05,0.16,0.08,0.00,0.04,0.00,0.06,0.00,0.11,0.47,0.33,0.00,0.18,0.0,0.00,0.46,0.01,0.00,...,0.0,0.01,0.0,0.01,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.01,0.0,0.0,0.01,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,3.27,1.48,0.49,0.04,0.12,0.21,0.17,0.00,0.16,1.02,0.96,0.01,0.41,0.00,0.00,0.38,1.32,0.16,0.33,0.00,1.13,0.22,0.00,0.39,0.66,0.00,0.00,0.00,0.85,0.07,0.02,1.31,3.38,0.00,0.94,0.0,0.00,0.90,0.20,0.10,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.02,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
5,1.49,1.16,0.47,0.09,0.69,0.59,0.21,0.09,0.00,0.30,1.19,0.01,0.06,0.00,0.00,1.14,0.35,0.10,0.34,0.00,0.36,0.00,0.08,0.03,0.29,0.00,0.02,0.01,0.03,0.00,0.08,0.48,1.10,0.00,0.64,0.0,0.00,1.54,0.01,0.01,...,0.0,0.03,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,1.61,0.77,0.40,0.13,0.34,1.30,0.32,0.04,0.20,1.15,0.94,0.11,0.04,0.57,0.08,1.12,1.82,0.67,0.58,0.11,1.45,0.70,0.21,0.14,2.01,0.29,0.11,0.68,1.06,0.69,0.50,2.34,1.98,0.88,1.98,0.0,0.01,1.33,0.11,0.73,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
668,0.20,0.02,0.04,0.02,0.02,0.11,0.03,0.00,0.02,0.02,0.02,0.01,0.01,0.04,0.00,0.21,0.12,0.03,0.00,0.00,0.06,0.03,0.00,0.06,0.18,0.01,0.00,0.04,0.10,0.04,0.01,0.49,0.12,0.04,0.32,0.0,0.00,0.04,0.01,0.04,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
669,0.30,0.01,0.09,0.02,0.08,0.26,0.06,0.00,0.14,0.07,0.19,0.03,0.00,0.03,0.01,0.37,0.10,0.02,0.18,0.00,0.06,0.01,0.00,0.33,0.08,0.01,0.06,0.01,0.02,0.01,0.03,0.18,0.13,0.01,0.09,0.0,0.00,0.13,0.01,0.02,...,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.01,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [19]:
cmat

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,39,40,41,...,155820,156025,156387,156607,156609,156726,157200,157296,157407,157667,158238,158314,158528,158956,159093,159462,159690,159755,159858,159972,160080,160271,160438,160440,160563,160565,160567,160590,160656,160718,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Create Application

In [36]:
def recomendation(uid,topk=5):
  res = list(reconstructed.T[uid].sort_values(ascending=False)[0:topk].index)
  res = moviedat[moviedat['movieId'].isin(res)]
  res['genre'] = [re.findall(r"name': '(.*?)'}", res) for res in res['genres']]
  mList = ratingdat[ratingdat['userId']==uid][['movieId','rating']]
  res = res.merge(mList,on='movieId',how='left')
  res = res[['movieId','title','genre','rating']]
  return res



Application Analysis

In [48]:
res = list(reconstructed.T[4].sort_values(ascending=False)[0:5].index)
res

[1196, 260, 1097, 1210, 1198]

In [49]:
res = moviedat[moviedat['movieId'].isin(res)]
res

,movieId,imdbId,tmdbId,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
232,260,76759,11.0,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,12110,tt0112896,en,Dracula: Dead and Loving It,When a lawyer shows up at the vampire's doorst...,5.43033,/xve4cgfYItnOhtzLYoTwTVy5FGr.jpg,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",1995-12-22,0.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Dracula: Dead and Loving It,False,5.7,210.0
888,1097,83866,601.0,False,"{'id': 43058, 'name': 'Homeward Bound Collecti...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 35, '...",NaN,25059,tt0116552,en,Homeward Bound II: Lost in San Francisco,Your favorite talking animals are back! The fi...,14.6578,/6oFXtbLt9IyoK6rdvdt0ccF6hGP.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1996-03-08,32709423.0,89.0,[],Released,Your favorite talking animals are back!,Homeward Bound II: Lost in San Francisco,False,6.0,105.0
953,1196,80684,1891.0,False,"{'id': 945, 'name': 'Lethal Weapon Collection'...",15000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",NaN,941,tt0093409,en,Lethal Weapon,Veteran buttoned-down LAPD detective Roger Mur...,13.1838,/drp80fBSWhz0vAmzUibXDyn0Pjo.jpg,"[{'name': 'Silver Pictures', 'id': 1885}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1987-03-06,120207127.0,110.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,If these two can learn to stand each other... ...,Lethal Weapon,False,7.0,1367.0
955,1198,82971,85.0,False,NaN,38000000,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,10534,tt0118158,en,White Squall,Teenage boys discover discipline and camarader...,9.5686,/3a5nomeyzJF4gG26HouCtce99ki.jpg,"[{'name': 'Hollywood Pictures', 'id': 915}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1996-02-02,10300000.0,129.0,"[{'iso_639_1': 'da', 'name': 'Dansk'}, {'iso_6...",Released,NaN,White Squall,False,6.3,86.0
966,1210,86190,1892.0,False,"{'id': 945, 'name': 'Lethal Weapon Collection'...",25000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,942,tt0097733,en,Lethal Weapon 2,"In the opening chase, Martin Riggs and Roger M...",12.8343,/7XAusKpFNv8x9sHpFLAS2lWCT8l.jpg,"[{'name': 'Silver Pictures', 'id': 1885}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1989-07-07,227853986.0,114.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The magic is back!,Lethal Weapon 2,False,6.7,1066.0


In [50]:
res['genre']=[re.findall(r"name': '(.*?)'}", res) for res in res['genres']]
res

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,movieId,imdbId,tmdbId,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,genre
232,260,76759,11.0,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,12110,tt0112896,en,Dracula: Dead and Loving It,When a lawyer shows up at the vampire's doorst...,5.43033,/xve4cgfYItnOhtzLYoTwTVy5FGr.jpg,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",1995-12-22,0.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Dracula: Dead and Loving It,False,5.7,210.0,"[Comedy, Horror]"
888,1097,83866,601.0,False,"{'id': 43058, 'name': 'Homeward Bound Collecti...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 35, '...",NaN,25059,tt0116552,en,Homeward Bound II: Lost in San Francisco,Your favorite talking animals are back! The fi...,14.6578,/6oFXtbLt9IyoK6rdvdt0ccF6hGP.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1996-03-08,32709423.0,89.0,[],Released,Your favorite talking animals are back!,Homeward Bound II: Lost in San Francisco,False,6.0,105.0,"[Adventure, Comedy, Drama, Family]"
953,1196,80684,1891.0,False,"{'id': 945, 'name': 'Lethal Weapon Collection'...",15000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",NaN,941,tt0093409,en,Lethal Weapon,Veteran buttoned-down LAPD detective Roger Mur...,13.1838,/drp80fBSWhz0vAmzUibXDyn0Pjo.jpg,"[{'name': 'Silver Pictures', 'id': 1885}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1987-03-06,120207127.0,110.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,If these two can learn to stand each other... ...,Lethal Weapon,False,7.0,1367.0,"[Adventure, Action, Comedy, Thriller, Crime]"
955,1198,82971,85.0,False,NaN,38000000,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,10534,tt0118158,en,White Squall,Teenage boys discover discipline and camarader...,9.5686,/3a5nomeyzJF4gG26HouCtce99ki.jpg,"[{'name': 'Hollywood Pictures', 'id': 915}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1996-02-02,10300000.0,129.0,"[{'iso_639_1': 'da', 'name': 'Dansk'}, {'iso_6...",Released,NaN,White Squall,False,6.3,86.0,"[Action, Drama]"
966,1210,86190,1892.0,False,"{'id': 945, 'name': 'Lethal Weapon Collection'...",25000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,942,tt0097733,en,Lethal Weapon 2,"In the opening chase, Martin Riggs and Roger M...",12.8343,/7XAusKpFNv8x9sHpFLAS2lWCT8l.jpg,"[{'name': 'Silver Pictures', 'id': 1885}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1989-07-07,227853986.0,114.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The magic is back!,Lethal Weapon 2,False,6.7,1066.0,"[Action, Adventure, Comedy, Crime, Thriller]"


In [47]:
res['genre']

232                                [Comedy, Horror]
888              [Adventure, Comedy, Drama, Family]
953    [Adventure, Action, Comedy, Thriller, Crime]
955                                 [Action, Drama]
966    [Action, Adventure, Comedy, Crime, Thriller]
Name: genre, dtype: object

In [52]:
mList = ratingdat[ratingdat['userId']==4][['movieId','rating']]
mList

,movieId,rating
147,10,4.0
148,34,5.0
149,112,5.0
150,141,5.0
151,153,4.0
...,...,...
346,3251,5.0
347,3255,4.0
348,3263,3.0
349,3265,5.0


In [53]:
res = res.merge(mList,on='movieId',how='left')
res

,movieId,imdbId,tmdbId,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,genre,rating
0,260,76759,11.0,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",NaN,12110,tt0112896,en,Dracula: Dead and Loving It,When a lawyer shows up at the vampire's doorst...,5.43033,/xve4cgfYItnOhtzLYoTwTVy5FGr.jpg,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",1995-12-22,0.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Dracula: Dead and Loving It,False,5.7,210.0,"[Comedy, Horror]",5.0
1,1097,83866,601.0,False,"{'id': 43058, 'name': 'Homeward Bound Collecti...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 35, '...",NaN,25059,tt0116552,en,Homeward Bound II: Lost in San Francisco,Your favorite talking animals are back! The fi...,14.6578,/6oFXtbLt9IyoK6rdvdt0ccF6hGP.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1996-03-08,32709423.0,89.0,[],Released,Your favorite talking animals are back!,Homeward Bound II: Lost in San Francisco,False,6.0,105.0,"[Adventure, Comedy, Drama, Family]",5.0
2,1196,80684,1891.0,False,"{'id': 945, 'name': 'Lethal Weapon Collection'...",15000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",NaN,941,tt0093409,en,Lethal Weapon,Veteran buttoned-down LAPD detective Roger Mur...,13.1838,/drp80fBSWhz0vAmzUibXDyn0Pjo.jpg,"[{'name': 'Silver Pictures', 'id': 1885}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1987-03-06,120207127.0,110.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,If these two can learn to stand each other... ...,Lethal Weapon,False,7.0,1367.0,"[Adventure, Action, Comedy, Thriller, Crime]",5.0
3,1198,82971,85.0,False,NaN,38000000,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,10534,tt0118158,en,White Squall,Teenage boys discover discipline and camarader...,9.5686,/3a5nomeyzJF4gG26HouCtce99ki.jpg,"[{'name': 'Hollywood Pictures', 'id': 915}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1996-02-02,10300000.0,129.0,"[{'iso_639_1': 'da', 'name': 'Dansk'}, {'iso_6...",Released,NaN,White Squall,False,6.3,86.0,"[Action, Drama]",5.0
4,1210,86190,1892.0,False,"{'id': 945, 'name': 'Lethal Weapon Collection'...",25000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,942,tt0097733,en,Lethal Weapon 2,"In the opening chase, Martin Riggs and Roger M...",12.8343,/7XAusKpFNv8x9sHpFLAS2lWCT8l.jpg,"[{'name': 'Silver Pictures', 'id': 1885}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1989-07-07,227853986.0,114.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The magic is back!,Lethal Weapon 2,False,6.7,1066.0,"[Action, Adventure, Comedy, Crime, Thriller]",5.0


In [54]:
res[['movieId','title','genre','rating']]

,movieId,title,genre,rating
0,260,Dracula: Dead and Loving It,"[Comedy, Horror]",5.0
1,1097,Homeward Bound II: Lost in San Francisco,"[Adventure, Comedy, Drama, Family]",5.0
2,1196,Lethal Weapon,"[Adventure, Action, Comedy, Thriller, Crime]",5.0
3,1198,White Squall,"[Action, Drama]",5.0
4,1210,Lethal Weapon 2,"[Action, Adventure, Comedy, Crime, Thriller]",5.0


## Application Usage

In [55]:
## comedy horror type
userid = 398
topk = 20
recomendation(userid,topk)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,movieId,title,genre,rating
0,110,The Tie That Binds,[Thriller],NaN
1,260,Dracula: Dead and Loving It,"[Comedy, Horror]",NaN
2,541,The Juror,"[Drama, Thriller]",NaN
3,589,New York Cop,"[Action, Thriller, Crime, Drama]",NaN
4,1036,Bhaji on the Beach,"[Comedy, Drama]",NaN
5,1097,Homeward Bound II: Lost in San Francisco,"[Adventure, Comedy, Drama, Family]",NaN
6,1136,The Fan,"[Drama, Mystery, Thriller]",4.0
7,1196,Lethal Weapon,"[Adventure, Action, Comedy, Thriller, Crime]",4.0
8,1197,Get Real,"[Drama, Romance]",5.0
9,1198,White Squall,"[Action, Drama]",4.0


In [56]:
## action thriller type
userid = 198
recomendation(userid,topk)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,movieId,title,genre,rating
0,50,Jack & Sarah,"[Comedy, Drama, Romance]",NaN
1,296,The Substitute,"[Action, Thriller]",NaN
2,318,Nell,"[Drama, Thriller]",NaN
3,593,Miami Rhapsody,"[Comedy, Romance]",NaN
4,608,My Family,"[Comedy, Drama]",3.5
5,858,Farinelli,"[History, Drama]",5.0
6,1089,North,"[Comedy, Drama, Family, Fantasy, Science Fiction]",2.0
7,1213,"I, the Worst of All","[Drama, Foreign]",NaN
8,2599,The Sea Inside,[Drama],5.0
9,2858,Cutthroat Island,"[Action, Adventure]",5.0


In [57]:
## comedy romance drama type
userid = 38
recomendation(userid,topk)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,movieId,title,genre,rating
0,50,Jack & Sarah,"[Comedy, Drama, Romance]",NaN
1,296,The Substitute,"[Action, Thriller]",5.0
2,318,Nell,"[Drama, Thriller]",4.5
3,356,Nixon,"[History, Drama]",4.5
4,527,Cabin Boy,[Comedy],4.5
5,593,Miami Rhapsody,"[Comedy, Romance]",4.5
6,858,Farinelli,"[History, Drama]",4.5
7,1193,The Remains of the Day,"[Drama, Romance]",NaN
8,1213,"I, the Worst of All","[Drama, Foreign]",4.0
9,1221,Gumby: The Movie,"[Fantasy, Animation, Science Fiction, Family]",4.5
